# Code of excess_est.R 

# Python traduction of Code 

In [1]:
# Need to install 
! pip install pandas 
! pip install pyreadr

### Imports 

In [2]:
import pandas as pd
import pyreadr
import math

### Loading resources

In [3]:
hh_main =  pyreadr.read_r("../data/rdata/hh_main.RDS")
deaths =  pyreadr.read_r("../data/rdata/deaths.RDS")
deaths_official =  pyreadr.read_r("../data/rdata/deaths_official.RDS")
weighted_pop_est = pyreadr.read_r("../data/rdata/pop_est.RDS")
adj_rates = pyreadr.read_r("../data/rdata/adj_rates.RDS")

hh_main = hh_main[None]
deaths = deaths[None]
deaths_official =  deaths_official[None]
weighted_pop_est = weighted_pop_est[None]
adj_rates = adj_rates[None]



In [4]:
deaths.head()

,hh_id,hh_id__1,died_month,died_b_p_hurricane,cause_of_death,cause_of_death_ot,mo,age
0,1061,4.0,12,---,0,---,12.0,81.0
1,1085,1.0,10,---,8,---,10.0,60.0
2,1387,2.0,11,---,8,---,11.0,101.0
3,1400,0.0,11,---,99,Infarto,11.0,54.0
4,1402,1.0,2,---,---,---,2.0,69.0


### Raw data

In [5]:
deaths_after_hurricane = len(deaths.loc[deaths['died_month'].isin(['10', '11', '12']) 
                                   | (deaths['died_month'].isin(['9']) & deaths['died_b_p_hurricane'].isin(['2']))])

In [6]:
deaths_before_hurricane = len(deaths.loc[deaths['died_month'].isin(['1', '2', '3', '4', '5', '6', '7', '8']) 
                                  | (deaths['died_month'].isin(['9']) & deaths['died_b_p_hurricane'].isin(['1']))])

In [7]:
total_pop = sum(hh_main['hh_size'])

adj_pop = (total_pop-deaths_before_hurricane) * (102/365)

rate = deaths_after_hurricane / adj_pop

se = math.sqrt(deaths_after_hurricane)/adj_pop

ll = rate - 1.96*se

ul = rate + 1.96*se


In [8]:
deaths_official

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Pop,Popv17
0,2010,2384,2218,2470,2290,2440,2405,2468,2564,2280,2468,2518,2776,3722000.0,3721525.0
1,2011,2714,2353,2755,2439,2472,2446,2451,2443,2364,2484,2347,2632,3679000.0,3678732.0
2,2012,2604,2442,2567,2480,2370,2560,2434,2467,2451,2425,2456,2579,3634000.0,3634488.0
3,2013,2558,2361,2693,2301,2381,2309,2462,2483,2457,2379,2254,2503,3593000.0,3593077.0
4,2014,2464,2191,2467,2373,2446,2371,2396,2521,2480,2866,2619,2858,3535000.0,3534874.0
5,2015,2711,2379,2395,2246,2322,2118,2361,2250,2235,2376,2245,2486,3473000.0,3473177.0
6,2016,2712,2562,2426,2225,2283,2322,2442,2405,2347,2341,2458,2817,3411307.0,3406520.0
7,2017,2862,2295,2469,2357,2376,2342,2342,2306,2887,2991,2571,2168,3337177.0,3337177.0


### To cacluate excess deaths from rate 

In [9]:
pop_in_2016 = 3406520
adj_pop_2016 = pop_in_2016*(102/365)

deaths_in_2016_temp = deaths_official.loc[deaths_official['Year'].isin(['2016'])]
deaths_in_2016_select = deaths_in_2016_temp[['Sep', 'Oct', 'Nov', 'Dec']]
deaths_in_2016 = deaths_in_2016_select.assign(Sep = deaths_in_2016_temp['Sep'].map(lambda x: x*(1/3)))
deaths_after_hurricane_2016 = deaths_in_2016.sum(axis = 1, skipna = True) 
rate_2016 = deaths_after_hurricane_2016/adj_pop_2016


### Rate difference

In [10]:
diff = rate - rate_2016
diff_ll = ll - rate_2016
diff_ul = ul - rate_2016 

### Excess deaths calculation

In [11]:
weighted_pop_est

,None
0,3.030307e+06


In [12]:
weighted_pop_est*(102/365)*diff
weighted_pop_est*(102/365)*diff_ll
weighted_pop_est*(102/365)*diff_ul

,None,6
0,NaN,NaN


### Adjusted calculations


In [13]:
adj_rate = round(adj_rates.rate_after,1) / 1000
adj_ll = round(adj_rates.lower_after,1) / 1000
adj_ul = round(adj_rates.upper_after,1) / 1000


### Excess deaths calculation

In [14]:
weighted_pop_est*(102/365)*(adj_rate-rate_2016)
weighted_pop_est*(102/365)*(adj_ll-rate_2016)
weighted_pop_est*(102/365)*(adj_ul-rate_2016)

,None,0,6
0,NaN,NaN,NaN
